# 第五章：格式化输出和为GPT预设响应

## 设置

运行以下设置单元格来加载您的API密钥并建立`get_completion`辅助函数。


In [1]:
# 安装OpenAI库
%pip install openai==1.61.0

# 🔧 OpenAI环境自动配置
# 此设置会自动从环境变量或IPython存储中加载配置

# 导入Python内置的正则表达式库
import re

# 🚀 使用统一配置管理系统
from config import setup_notebook_environment, print_config_info

# 自动设置OpenAI客户端和get_completion函数
# 优先级：环境变量 > IPython存储 > 默认值
try:
    client, get_completion = setup_notebook_environment()
    print("✅ 使用统一配置管理成功！")
except Exception as e:
    print(f"❌ 统一配置失败，回退到传统方式: {e}")
    
    # 回退到传统的配置方式
    import openai
    
    # 从IPython存储中检索API_KEY和MODEL_NAME变量
    %store -r API_KEY
    %store -r MODEL_NAME

    # 如果没有设置MODEL_NAME，使用默认值
    try:
        MODEL_NAME
    except NameError:
        MODEL_NAME = "gpt-4o"  # 默认使用gpt-4o模型

    # 创建OpenAI客户端
    client = openai.OpenAI(api_key=API_KEY)

    # 新增了prefill参数用于预填充文本，默认值为空字符串
    def get_completion(prompt: str, system_prompt="", prefill=""):
        """
        获取GPT的完成响应
        
        参数:
            prompt (str): 用户提示内容
            system_prompt (str): 系统提示（可选）
            prefill (str): 预填充文本（可选），用于引导GPT的响应开始
        
        返回:
            str: GPT的响应文本
        """
        # 构建消息列表
        messages = []
        
        # 如果有系统提示，添加系统消息
        if system_prompt:
            messages.append({"role": "system", "content": system_prompt})
        
        # 添加用户消息
        messages.append({"role": "user", "content": prompt})
        
        # 如果有预填充内容，添加助手消息
        if prefill:
            messages.append({"role": "assistant", "content": prefill})
        
        # 调用OpenAI API
        response = client.chat.completions.create(
            model=MODEL_NAME,              # 使用的模型名称
            messages=messages,             # 消息列表
            max_tokens=2000,              # 最大生成的token数量
            temperature=0.0               # 温度参数，0表示更确定性的回答
        )
        
        # 如果有预填充内容，将其与生成的内容组合
        if prefill:
            return prefill + response.choices[0].message.content
        else:
            return response.choices[0].message.content
    
    print("⚠️  使用传统配置方式，建议配置环境变量以获得更好体验")


Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Note: you may need to restart the kernel to use updated packages.
✅ OpenAI环境设置完成!
🔧 OpenAI API 配置信息:
  📡 配置来源: 环境变量 + 自定义API地址: https://vip.apiyi.com/v1
  🤖 模型: gpt-4o
  🌐 API地址: https://vip.apiyi.com/v1
  🔑 API密钥: sk-R2utG...B944

✅ 使用统一配置管理成功！


---

## 课程

**GPT可以以多种方式格式化其输出**。您只需要明确要求它这样做！

其中一种方式是使用XML标签将响应与其他多余文本分离。您已经学会了可以使用XML标签使您的提示对GPT更清晰、更易解析。事实证明，您还可以要求GPT**使用XML标签使其输出对人类更清晰、更容易理解**。

### 技术要点

1. **XML标签格式化**：指导GPT使用结构化的XML标签来组织输出
2. **预设响应（Prefill）**：通过在assistant角色中预填充部分内容来引导GPT的响应方向
3. **结构化输出**：要求GPT按照特定的格式（如JSON、表格、列表等）输出内容
4. **多段落组织**：使用标记来分离不同类型的信息

### 示例 1：使用XML标签格式化输出

要求GPT使用XML标签来组织其回答，使输出更加结构化和易于解析。


### 示例 1：使用XML标签格式化输出

要求GPT使用XML标签来组织其回答，使输出更加结构化和易于解析。


In [2]:
# 示例1：XML标签格式化输出
prompt = """
请分析人工智能对教育行业的影响，并使用以下XML标签格式化你的回答：

<分析>
在这里提供主要分析内容
</分析>

<优势>
列出AI在教育中的优势
</优势>

<挑战>
列出AI在教育中面临的挑战
</挑战>

<建议>
提供实施建议
</建议>
"""

response = get_completion(prompt)
print("🤖 GPT回答:")
print(response)


🤖 GPT回答:
<分析>
人工智能在教育行业的影响是深远且多方面的。AI技术可以通过个性化学习、自动化行政任务和增强教学资源来改变传统教育模式。个性化学习使得学生能够按照自己的节奏和学习风格进行学习，而自动化行政任务则减轻了教师的负担，使他们能够专注于教学本身。此外，AI还可以通过分析大量数据来提供洞察，从而帮助教育机构优化课程设置和教学方法。
</分析>

<优势>
1. 个性化学习：AI可以根据学生的学习习惯和进度提供定制化的学习计划。
2. 提高效率：自动化任务如评分和考勤管理可以节省教师的时间。
3. 数据分析：AI可以分析学生表现数据，帮助教师识别学习困难并提供针对性支持。
4. 增强资源：通过虚拟现实和增强现实技术，AI可以提供更丰富的学习体验。
</优势>

<挑战>
1. 隐私问题：学生数据的收集和使用可能引发隐私和安全问题。
2. 技术依赖：过度依赖技术可能导致教师和学生的互动减少。
3. 资源不均：不同地区和学校可能无法平等地获得AI技术。
4. 伦理问题：AI决策可能缺乏人类的伦理判断，影响教育公平性。
</挑战>

<建议>
1. 加强数据保护：制定严格的数据保护政策，确保学生信息的安全。
2. 平衡技术使用：在教学中合理使用AI技术，确保人际互动的重要性。
3. 提供培训：为教师和学生提供AI技术的培训，以提高其使用效率。
4. 促进公平：政府和教育机构应努力缩小技术资源的差距，确保所有学生都能受益于AI技术。
</建议>


### 示例 2：预设响应（Prefill）技术

预设响应是一种强大的技术，可以：
- 引导GPT的响应风格和格式
- 确保输出的一致性
- 控制回答的开始部分

我们将使用 `get_completion` 函数的 `prefill` 参数来演示这个功能。


In [3]:
# 示例2：使用预设响应引导GPT风格
prompt = """
作为一名资深的Python开发专家，请解释什么是装饰器，并提供一个实用的示例。
"""

# 使用预设响应来引导GPT以特定的风格和格式回答
prefill = """## 🐍 Python装饰器详解

### 概念理解
装饰器是Python中的一个重要概念，"""

response = get_completion(prompt, prefill=prefill)
print("🤖 使用预设响应的回答:")
print(response)
print("\n" + "="*60 + "\n")

# 对比：不使用预设响应的回答
response_normal = get_completion(prompt)
print("🤖 普通回答:")
print(response_normal)


🤖 使用预设响应的回答:
## 🐍 Python装饰器详解

### 概念理解
装饰器是Python中的一个重要概念，好的，继续解释。

### 什么是装饰器？

装饰器是一个用于修改函数或方法行为的函数。它允许在不改变原函数代码的情况下，动态地增加功能。装饰器通常用于横切关注点（cross-cutting concerns），如日志记录、权限验证、性能测试、事务处理等。

### 装饰器的基本结构

装饰器本质上是一个高阶函数，它接收一个函数作为参数，并返回一个新的函数。装饰器通常使用Python的`@`语法糖来应用。

```python
def my_decorator(func):
    def wrapper(*args, **kwargs):
        # 在调用原函数之前可以执行一些操作
        print("Something is happening before the function is called.")
        
        # 调用原函数
        result = func(*args, **kwargs)
        
        # 在调用原函数之后可以执行一些操作
        print("Something is happening after the function is called.")
        
        return result
    return wrapper
```

### 使用装饰器

可以通过在函数定义前加上`@装饰器名`来应用装饰器。

```python
@my_decorator
def say_hello(name):
    print(f"Hello, {name}!")

# 调用函数
say_hello("Alice")
```

### 实用示例：计算函数执行时间

下面是一个实用的装饰器示例，用于计算函数的执行时间：

```python
import time

def timer_decorator(func):
    def wrapper(*args, **kwargs):
        start_time = time.time()  # 记录开始时间
        result = func(*ar

### 示例 3：JSON格式输出

要求GPT输出标准的JSON格式，便于程序解析和处理。


In [13]:
# 示例3：JSON格式输出
prompt = """
请为一个在线书店设计一个图书数据结构，包含以下信息：
- 书名
- 作者
- ISBN
- 价格
- 分类
- 出版日期
- 库存数量
- 评分

请以标准JSON格式输出，并提供3本示例图书的数据。

输出格式要求：
1. 使用中文字段名
2. 价格用数字类型
3. 评分范围0-5
4. 出版日期格式：YYYY-MM-DD
"""

response = get_completion(prompt)
print("🤖 JSON格式输出:")
print(response)

🤖 JSON格式输出:
```json
[
    {
        "书名": "活着",
        "作者": "余华",
        "ISBN": "9787506365437",
        "价格": 39.00,
        "分类": "文学",
        "出版日期": "2012-08-01",
        "库存数量": 120,
        "评分": 4.8
    },
    {
        "书名": "三体",
        "作者": "刘慈欣",
        "ISBN": "9787536692930",
        "价格": 45.00,
        "分类": "科幻",
        "出版日期": "2008-01-01",
        "库存数量": 200,
        "评分": 4.9
    },
    {
        "书名": "解忧杂货店",
        "作者": "东野圭吾",
        "ISBN": "9787544270878",
        "价格": 38.00,
        "分类": "小说",
        "出版日期": "2014-05-01",
        "库存数量": 150,
        "评分": 4.7
    }
]
```


### 示例 4：表格格式输出

要求GPT以表格形式输出数据，便于阅读和比较。


In [5]:
# 示例4：表格格式输出
prompt = """
请比较Python、Java、Go三种编程语言在以下方面的特点：
- 性能表现
- 学习难度  
- 开发效率
- 生态系统
- 应用场景

请以Markdown表格格式输出比较结果，每个维度用1-5分评分（5分最高）。
"""

response = get_completion(prompt)
print("🤖 表格格式输出:")
print(response)


🤖 表格格式输出:
以下是Python、Java和Go在性能表现、学习难度、开发效率、生态系统和应用场景方面的比较：

| 特点         | Python | Java | Go   |
|--------------|--------|------|------|
| 性能表现     | 3      | 4    | 4    |
| 学习难度     | 2      | 3    | 3    |
| 开发效率     | 5      | 3    | 4    |
| 生态系统     | 5      | 5    | 4    |
| 应用场景     | 5      | 5    | 4    |

### 详细说明

- **性能表现**: 
  - **Python**: 由于其解释型语言的特性，Python的运行速度通常较慢，但可以通过C扩展和JIT编译器（如PyPy）提高性能。
  - **Java**: Java通过JIT编译和JVM优化，通常具有较好的性能表现。
  - **Go**: Go语言编译为机器码，启动速度快，性能接近C/C++。

- **学习难度**:
  - **Python**: 语法简洁，易于上手，适合初学者。
  - **Java**: 语法相对复杂，面向对象编程概念较多，学习曲线稍陡。
  - **Go**: 语法简单，设计上避免了复杂特性，学习难度适中。

- **开发效率**:
  - **Python**: 丰富的库和框架支持，开发效率高，适合快速原型开发。
  - **Java**: 开发效率中等，强类型系统和丰富的工具支持有助于大型项目的开发。
  - **Go**: 简单的语法和高效的并发支持提高了开发效率，适合开发高性能网络服务。

- **生态系统**:
  - **Python**: 拥有广泛的库和框架支持，特别是在数据科学、机器学习和Web开发领域。
  - **Java**: 拥有成熟的生态系统，广泛应用于企业级应用开发。
  - **Go**: 生态系统正在快速发展，特别是在云计算和微服务领域。

- **应用场景**:
  - **Python**: 广泛应用于数据科学、人工智能、Web开发、自动化脚本等领域。
  - **Java**: 主要用于企业级应用、Android开

In [6]:
# 示例5：多步骤思维过程输出
prompt = """
一个初创公司计划开发一个AI客服系统，预算30万元，团队3人，开发周期6个月。
请帮助分析项目的可行性，并按照以下格式组织你的分析：

<问题理解>
对项目需求的理解和关键问题识别
</问题理解>

<技术分析>
技术方案和实现难度分析
</技术分析>

<资源评估>
人力、资金、时间资源的合理性评估
</资源评估>

<风险识别>
潜在的技术风险和业务风险
</风险识别>

<可行性结论>
综合评估结果和建议
</可行性结论>

<实施建议>
具体的实施计划和里程碑
</实施建议>
"""

response = get_completion(prompt)
print("🤖 结构化分析输出:")
print(response)


🤖 结构化分析输出:
<问题理解>
该项目旨在开发一个AI客服系统，预算为30万元，团队由3人组成，开发周期为6个月。关键问题包括：如何在有限的预算和时间内开发出一个功能完善且稳定的AI客服系统，以及如何确保系统能够满足客户的需求并提供良好的用户体验。

</问题理解>

<技术分析>
技术方案可能包括自然语言处理（NLP）、机器学习模型训练、对话管理系统以及与现有客服系统的集成。实现难度主要在于NLP的准确性和对话管理的智能化程度。选择合适的开源工具和平台（如TensorFlow、PyTorch、Rasa等）可以降低开发难度。需要评估团队成员的技术能力，以确保他们能够胜任这些技术任务。

</技术分析>

<资源评估>
人力资源方面，3人团队需要具备多方面的技能，包括NLP、软件开发、项目管理等。资金方面，30万元的预算需要合理分配到人员工资、技术工具、服务器租赁等方面。时间资源方面，6个月的开发周期较为紧张，需要制定详细的项目计划以确保按时完成。

</资源评估>

<风险识别>
技术风险包括NLP模型的准确性不足、系统集成困难、性能优化问题等。业务风险包括市场需求变化、客户反馈不佳、竞争对手的压力等。需要提前制定应对策略，如技术迭代计划和市场调研。

</风险识别>

<可行性结论>
综合评估，该项目在预算和时间上存在一定挑战，但通过合理的技术选择和资源分配，可以实现目标。建议在项目初期进行详细的需求分析和技术评估，以确保项目的可行性。

</可行性结论>

<实施建议>
1. 项目启动（第1个月）：进行详细的需求分析和技术方案设计，确定使用的技术工具和平台。
2. 原型开发（第2-3个月）：开发系统原型，进行初步测试和迭代。
3. 系统集成（第4个月）：将AI客服系统与现有客服系统进行集成，确保数据流畅。
4. 测试与优化（第5个月）：进行全面的系统测试，优化性能和用户体验。
5. 项目交付（第6个月）：完成最终版本的开发，进行用户培训和系统部署。

</实施建议>


##### 示例 5：代码输出格式控制

控制GPT输出代码时的格式和注释风格。


In [7]:
# 示例5：代码输出格式控制
prompt = """
请实现一个Python类来管理学生信息，包含添加、删除、查询、更新功能。

输出要求：
1. 使用以下格式组织代码：
   <代码说明>
   简要说明类的功能和设计思路
   </代码说明>
   
   <代码实现>
   ```python
   # 代码内容
   ```
   </代码实现>
   
   <使用示例>
   ```python
   # 示例代码
   ```
   </使用示例>

2. 代码注释要求：
   - 类和方法都要有中文文档字符串
   - 关键逻辑添加行内注释
   - 符合PEP 8规范
"""

response = get_completion(prompt)
print("🤖 格式化代码输出:")
print(response)


🤖 格式化代码输出:
<代码说明>
该Python类用于管理学生信息，提供添加、删除、查询和更新功能。设计思路是使用一个字典来存储学生信息，其中键为学生ID，值为学生的详细信息（如姓名、年龄、成绩等）。类提供了相应的方法来操作这个字典，以实现对学生信息的管理。
</代码说明>

<代码实现>
```python
class StudentManager:
    """
    学生信息管理类，提供添加、删除、查询和更新学生信息的功能。
    """

    def __init__(self):
        """初始化学生信息管理器，创建一个空的学生信息字典。"""
        self.students = {}

    def add_student(self, student_id, name, age, grade):
        """
        添加学生信息。

        :param student_id: 学生ID
        :param name: 学生姓名
        :param age: 学生年龄
        :param grade: 学生成绩
        """
        if student_id in self.students:
            raise ValueError("学生ID已存在。")
        self.students[student_id] = {'name': name, 'age': age, 'grade': grade}

    def remove_student(self, student_id):
        """
        删除学生信息。

        :param student_id: 学生ID
        """
        if student_id not in self.students:
            raise ValueError("学生ID不存在。")
        del self.students[student_id]

    def update_student(self, student_id, name=None, age=None, grade=Non

In [8]:
# 示例7：预设响应实现特定风格
prompt = """
解释什么是微服务架构，它解决了什么问题，有哪些优缺点？
"""

# 预设架构师风格的回答开头
prefill = """🏗️ **微服务架构深度解析**

作为一名拥有15年经验的解决方案架构师，我来为您详细解析微服务架构：

## 📋 核心概念

微服务架构（Microservices Architecture）是一种"""

response = get_completion(prompt, prefill=prefill)
print("🤖 架构师风格的专业回答:")
print(response)
print("\n" + "="*60 + "\n")

# 对比：预设技术博主风格
prefill_blogger = """嘿！今天我们来聊聊微服务这个热门话题！🔥

作为一个踩过无数坑的开发者，我想用最接地气的方式给大家讲讲微服务到底是个啥：

## 什么是微服务？

说白了，微服务就是"""

response_blogger = get_completion(prompt, prefill=prefill_blogger)
print("🤖 技术博主风格的回答:")
print(response_blogger)


🤖 架构师风格的专业回答:
🏗️ **微服务架构深度解析**

作为一名拥有15年经验的解决方案架构师，我来为您详细解析微服务架构：

## 📋 核心概念

微服务架构（Microservices Architecture）是一种微服务架构（Microservices Architecture）是一种软件设计风格，它将应用程序拆分为一组小型、独立部署和管理的服务。这些服务通常围绕业务功能构建，并通过轻量级的通信协议（通常是HTTP/REST或消息队列）进行交互。

## 🛠️ 解决的问题

1. **单体应用的复杂性**：传统的单体应用随着功能的增加变得庞大且复杂，难以维护和扩展。微服务通过拆分应用程序，降低了单个服务的复杂性。

2. **部署和扩展困难**：单体应用需要整体部署，微服务允许独立部署和扩展各个服务，提升了灵活性和效率。

3. **技术栈限制**：单体应用通常限制在一个技术栈内，微服务允许不同服务使用最适合的技术栈，促进技术创新。

4. **故障隔离**：单体应用中一个模块的故障可能影响整个系统，微服务架构通过服务隔离减少故障影响。

## ✅ 优点

1. **灵活性和可扩展性**：可以根据需求独立扩展某个服务，而不影响其他服务。

2. **技术多样性**：不同服务可以使用不同的技术栈，适应不同的业务需求。

3. **快速部署**：独立的服务可以更快地开发、测试和部署，缩短交付周期。

4. **故障隔离**：服务之间的隔离减少了故障传播，提高了系统的稳定性。

5. **团队独立性**：不同团队可以负责不同的服务，促进团队的独立性和专注度。

## ❌ 缺点

1. **复杂性增加**：管理多个服务的复杂性增加，需要有效的服务协调和监控。

2. **网络开销**：服务间通信增加了网络开销和延迟，可能影响性能。

3. **数据一致性**：分布式系统中的数据一致性问题需要额外的设计和管理。

4. **部署和运维挑战**：需要成熟的CI/CD流程和运维工具来管理服务的部署和监控。

5. **安全性**：服务间通信需要额外的安全措施来保护数据和服务。

微服务架构适合那些需要快速迭代、灵活扩展和技术多样性的应用场景，但也需要成熟的团队和工具来管理其复杂性和运维挑战。


🤖 技术博主风格的回答:
嘿！今天我们来聊聊微服务这个热门话题！🔥


### 实验 1：自定义XML标签格式

尝试设计你自己的XML标签结构来组织特定类型的输出。


In [9]:
# 实验1：自定义XML标签格式
# 在这里尝试你自己的格式化提示词

custom_prompt = """
# 请在这里输入你的提示词，尝试使用自定义的XML标签格式
# 例如：分析一个技术趋势、评估一个产品、制定一个计划等
"""

# 取消下面的注释来运行你的实验
# response = get_completion(custom_prompt)
# print("🔬 你的实验结果:")
# print(response)

print("💡 提示：修改上面的 custom_prompt 变量，然后取消注释运行代码")


💡 提示：修改上面的 custom_prompt 变量，然后取消注释运行代码


### 实验 2：预设响应风格测试

尝试不同的预设响应来改变GPT的回答风格和格式。


In [10]:
# 实验2：预设响应风格测试
# 选择一个你感兴趣的主题
your_prompt = "请解释区块链技术的工作原理"

# 尝试不同的预设响应风格
prefill_styles = {
    "学术风格": "# 区块链技术原理分析\n\n## 摘要\n\n区块链作为一种分布式账本技术，",
    "科普风格": "👋 大家好！今天我们来聊聊区块链这个神奇的技术！\n\n想象一下，",
    "技术文档风格": "## 区块链技术文档\n\n### 1. 概述\n\n区块链（Blockchain）是",
    "问答风格": "Q: 什么是区块链技术？\nA: 区块链是一种",
}

print("🎨 不同风格的预设响应效果对比：")
print("="*60)

for style_name, prefill in prefill_styles.items():
    print(f"\n📝 {style_name}:")
    print("-" * 30)
    response = get_completion(your_prompt, prefill=prefill)
    # 只显示前200个字符作为预览
    preview = response[:200] + "..." if len(response) > 200 else response
    print(preview)
    print()

# 你也可以尝试自己的预设响应
print("\n🔧 尝试你自己的预设响应:")
your_prefill = """
# 在这里输入你想要的预设响应开头
"""

# 取消注释来测试你的预设响应
# your_response = get_completion(your_prompt, prefill=your_prefill)
# print("🎯 你的预设响应效果:")
# print(your_response)


🎨 不同风格的预设响应效果对比：

📝 学术风格:
------------------------------
# 区块链技术原理分析

## 摘要

区块链作为一种分布式账本技术，当然，以下是对区块链技术工作原理的详细解释：

## 1. 基本概念

### 1.1 区块
区块链由一系列区块组成，每个区块包含若干交易记录。每个区块包括一个区块头和区块体。区块头包含元数据，如时间戳、前一个区块的哈希值和当前区块的哈希值。

### 1.2 链
区块通过加密哈希函数链接在一起，形成一个链条。每个区块的哈希值是...


📝 科普风格:
------------------------------
👋 大家好！今天我们来聊聊区块链这个神奇的技术！

想象一下，当然！区块链技术可以被视为一个分布式的数字账本，它记录了所有的交易信息，并且这些信息是不可更改的。以下是区块链技术的基本工作原理：

1. **分布式网络**：区块链是一个分布式系统，意味着它的账本不是存储在一个单独的中央服务器上，而是分布在多个节点（计算机）上。这些节点共同维护和更新账本。

2. **区块**：交易信息被打包成一个个...


📝 技术文档风格:
------------------------------
## 区块链技术文档

### 1. 概述

区块链（Blockchain）是### 2. 基本原理

区块链是一种分布式账本技术，允许多个参与者在没有中央权威的情况下共同维护一个数据记录。其核心思想是通过密码学和共识机制来确保数据的安全性和一致性。

### 3. 结构

区块链由一系列区块组成，每个区块包含以下几个部分：

- **区块头（Block Header）**：包括前一个区块的哈希值...


📝 问答风格:
------------------------------
Q: 什么是区块链技术？
A: 区块链是一种分布式账本技术，它通过去中心化和加密算法来确保数据的安全性和透明性。区块链由一系列称为“区块”的数据结构组成，这些区块按时间顺序链接在一起，形成一个链条。

Q: 区块链如何工作？
A: 区块链的工作原理可以分为以下几个关键步骤：

1. **交易记录**：当一笔交易发生时，它会被记录为一个数据块。这个数据块包含交易的详细信息，如时间戳、参与方和交易金额...




### 实验 3：复合格式输出

结合多种格式化技术，创建复杂的结构化输出。


In [11]:
# 实验3：复合格式输出
prompt = """
请为一个新的社交媒体应用制定产品规划，要求：

1. 使用XML标签组织内容结构
2. 在技术部分使用表格格式
3. 在时间规划部分使用JSON格式
4. 包含代码示例使用代码块格式

具体要求：
<产品概述>
简要描述产品定位和核心功能
</产品概述>

<目标用户>
用户画像分析
</目标用户>

<功能规划>
核心功能列表和优先级（用表格格式）
</功能规划>

<技术架构>
技术选型对比表格（包含前端、后端、数据库等）
</技术架构>

<开发计划>
6个月的开发时间线（JSON格式，包含阶段、任务、时间等字段）
</开发计划>

<API示例>
提供用户注册接口的代码示例
</API示例>
"""

# 使用预设响应确保格式正确
prefill = """📱 **社交媒体应用产品规划**

<产品概述>
本产品是一个面向年轻用户的"""

response = get_completion(prompt, prefill=prefill)
print("🔄 复合格式化输出:")
print(response)


🔄 复合格式化输出:
📱 **社交媒体应用产品规划**

<产品概述>
本产品是一个面向年轻用户的<产品概述>
本产品是一个面向年轻用户的社交媒体应用，旨在提供一个互动性强、内容丰富的平台。核心功能包括实时聊天、动态分享、兴趣小组、以及个性化推荐系统。用户可以通过应用发现新朋友、分享生活点滴、参与兴趣讨论，并获得个性化的内容推荐。
</产品概述>

<目标用户>
目标用户主要是18-35岁的年轻人，他们对社交互动有较高的需求，喜欢分享生活和参与社区活动。用户画像包括：
- 年轻的都市白领
- 大学生
- 社交活跃的创意人士
- 热衷于探索新事物的用户
</目标用户>

<功能规划>
| 功能名称       | 描述                     | 优先级 |
|----------------|--------------------------|--------|
| 实时聊天       | 提供一对一和群组聊天功能 | 高     |
| 动态分享       | 用户可以发布文字、图片和视频动态 | 高     |
| 兴趣小组       | 用户可以加入或创建兴趣小组 | 中     |
| 个性化推荐系统 | 基于用户行为推荐内容     | 中     |
| 用户资料管理   | 用户可以编辑个人资料     | 低     |
</功能规划>

<技术架构>
| 技术组件 | 选型对比                          |
|----------|----------------------------------|
| 前端     | React vs Vue.js                  |
| 后端     | Node.js vs Django                |
| 数据库   | MongoDB vs PostgreSQL             |
| 实时通信 | WebSocket vs Firebase Realtime DB |
</技术架构>

<开发计划>
```json
[
  {
    "阶段": "需求分析",
    "任务": "确定核心功能和用户需求",
    "时间": "2023-11-01 至 2023-11-15"
  },
  {
    